In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

!pip install tensorflow-addons
import tensorflow_addons as tfa
!pip install --quiet vit-keras
import tensorflow_addons as tfa
from vit_keras import vit
from keras.applications.vgg19 import preprocess_input
import pandas as pd
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 9.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


  Preparing metadata (setup.py) ... done
TensorFlow Version 2.12.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Step 7: Preprocessing the Data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   preprocessing_function = preprocess_input)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function = preprocess_input)

# Step 8: Load the Data
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/face_emotions/train',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/face_emotions/test',
                                            target_size = (224, 224),
                                            batch_size = 128,
                                            class_mode = 'categorical')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:

# Image size

IMAGE_SIZE = (224,224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
BATCH_SIZE = 64
EPOCHS = 20
ADAM_LEARNING_RATE = 0.0001
PATIENCE =10
classes=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']


In [ ]:

vit_model = vit.vit_b32(
        image_size = IMAGE_SIZE,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 7)
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(7, 'softmax')
    ],
    name = 'vision_transformer')

model.summary()


Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b32 (Functional)        (None, 768)               87455232  
                                                                 
 flatten_1 (Flatten)         (None, 768)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 768)              3072      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 128)               98432     
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 7)          

In [ ]:
learning_rate = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

METRICS = ['accuracy']
model.compile(optimizer = optimizer,
              loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2),
              metrics = METRICS)




In [ ]:
model.fit(x = training_set,
          validation_data =test_set,
          epochs = EPOCHS,
          )


In [ ]:
model_json = model.to_json()
with open("ViT_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("ViT_model.h5")
print("Saved model to disk")
from google.colab import files
files.download('ViT_model.h5')
files.download('ViT_model.json')